Небольшой скрипт для быстрого создания датасета по данным выбранных стран, городов или станций. Использует данные, скачанные через модуль aqreport_loader написанный Александрой Насоновой. Скрипт сканирует текущую директорию, далее создает список всех csv файлов, вложенных в папку data. Далее полученный список файлов фильтруется исходя из заданных параметров одной из трех выбранных функций и итоговый short-list файлов объединяется в один единый датасет, который сохраняется в текущей директории. При запуске скрипта необходимо удостовериться, что текущая директория содержит файл с новыми метаданными и папку с исходными данными.

In [1]:
import os
import time
import glob
import pathlib
import pandas as pd

In [2]:
csv1 = glob.glob(os.getcwd() + '/data' + '/**/*.csv', recursive=True)
codes = [] 
for file in csv1:
    codes.append(file.split('\\')[-1][:2])
codes = list(set(codes))
print(codes)

['GI', 'NL', 'RS', 'LV', 'HU', 'XK', 'NO', 'CZ', 'ES', 'RO', 'EE', 'LT', 'PT', 'GR', 'GE', 'BG', 'CH', 'UA', 'FR', 'PL', 'DE', 'TR', 'ME', 'SE', 'FI', 'MT', 'GB', 'AL', 'SK', 'CY', 'AD', 'MK', 'IT', 'BA', 'BE', 'LU', 'AT', 'IS', 'IE', 'SI', 'DK', 'HR']


In [3]:
def dataset_by_country(*codes) -> None:  ### pass citycodes separated by comma
    codelist = [*codes]
    csv2 = []
    for file in csv1:
        for code in codelist:
            if code in file:
                csv2.append(file)
    df = pd.concat([pd.read_csv(f) for f in csv2])
    df.to_csv( "df_" + '_'.join(code for code in codelist) + ".csv", index=False, encoding='utf-8-sig')

In [4]:
def dataset_by_localid(*codes) -> None:  ### pass localcodes separated by comma
    codelist = [*codes]
    csv3 = []
    for file in csv1:
        if file.split('_')[2] in codelist:
            csv3.append(file)
    df = pd.concat([pd.read_csv(f) for f in csv3])
    df.to_csv( "df_" + '_'.join(code for code in codelist) + ".csv", index=False, encoding='utf-8-sig')

In [5]:
def dataset_by_city(*names) -> None:     ### pass citynames separated by comma
    dfm = pd.read_csv("final_metadata.csv", low_memory=False)
    citynames = [*names]
    localids = []
    csv4 = []
    cities_lower = []
    cities_regular = []
    for city in list(dfm.query('StationCity==StationCity')['StationCity'].unique()):
        cities_lower.append(city.lower().split(' ')[0])
        cities_regular.append(city)
    for i in range(len(cities_lower)):
        for cityname in citynames:
            if cityname.lower() in cities_lower[i]:
                localids.extend(list(dfm[(dfm['StationCity']==cities_regular[i]) & (dfm['LocalCode'].notna())]['LocalCode'].unique()))                  
    for ele in localids:
        for file in csv1:
            if int(ele) == int(file.split('_')[2]):
                csv4.append(file)
    df = pd.concat([pd.read_csv(f) for f in csv4])
    df.to_csv( "df_" + '_'.join(cityname for citynames in citynames) + ".csv", index=False, encoding='utf-8-sig')

In [6]:
# dataset_by_country('DK', 'RS')

In [7]:
# dataset_by_city('Paris')